<a href="https://colab.research.google.com/github/datamilas/CroLyricsProject/blob/master/Python/google_colab/create_databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from os import path


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
##create info db
conn = sqlite3.connect('info.db')
c = conn.cursor()
c.execute("""CREATE TABLE artists (
            code text,
            name text
            )""")

c.execute("""CREATE TABLE songs_info (
            code text,
            name text,
            artist_code text,
            artist_name text,
            views integer,
            lyrics_by text,
            music_by text,
            year integer,
            album text,
            duration text
            )""")


In [ ]:
def insert_artist(artist_code, artist_name):
    with conn:
        c.execute("INSERT INTO artists VALUES (:code, :name)", {'code': artist_code, 'name': artist_name})

def insert_song_info(song_code, song_name, artist_code, artist_name, views, lyrics_by, music_by, year, album, duration):
    with conn:
        c.execute("INSERT INTO songs_info VALUES (:code, :name, :artist_code, :artist_name, :views, :lyrics_by, :music_by, :year, :album, :duration)", 
                  {'code': song_code, 'name': song_name, 'artist_code': artist_code, 'artist_name': artist_name, 'views': views, 'lyrics_by': lyrics_by, 'music_by': music_by, 'year': year, 'album': album, 'duration': duration})

In [ ]:
df_info = pd.read_csv(f"/content/gdrive/My Drive/CroLyrics_data/info_for_scraping.csv")

In [ ]:
for artist_name in df_info.artist_name:
  file_path = f"/content/gdrive/My Drive/CroLyrics_data/{artist_name}_clean_info.csv"
  if path.exists(file_path):
    df = pd.read_csv(file_path)
    artist_code = df.Song_ID[0].split("_")[0]
    insert_artist(artist_code, artist_name)

    for row in df.iterrows():
      info = row[1]
      insert_song_info(info.Song_ID, info.Song, artist_code, artist_name, info.Views, info.Lyrics_by, info.Music_by, info.Year, info.Album, info.Song_duration_discogs)

In [15]:
df = pd.DataFrame()
df_lyrics = pd.DataFrame()

for artist_name in df_info.artist_name:
  nlp_file_path = f"/content/gdrive/My Drive/CroLyrics_data/{artist_name}_nlp.csv"
  if path.exists(nlp_file_path):
    df_nlp = pd.read_csv(nlp_file_path)
    
    df_lyrics = df_lyrics.append(pd.read_csv(f"/content/gdrive/My Drive/CroLyrics_data/{artist_name}_final_lyrics.csv"))
    df = df.append(df_nlp)
    df = df[df["lemma"] != "´"].reset_index(drop=True)
    df["lemma"] = df.lemma.astype(str)

df_lyrics.to_csv(f"/content/gdrive/My Drive/CroLyrics_data/final_lyrics_all.csv", index=False)
df.to_csv(f"/content/gdrive/My Drive/CroLyrics_data/nlp_all.csv", index=False)

AttributeError: ignored

In [17]:
df_embedded = pd.DataFrame()

for artist_name in df_info.artist_name:
  file_path = f"/content/gdrive/My Drive/CroLyrics_data/{artist_name}_final_lyrics_embedded.csv"
  if path.exists(file_path):
    df_embedded = df_embedded.append(pd.read_csv(file_path))

df_embedded.to_csv(f"/content/gdrive/My Drive/CroLyrics_data/embedded_all.csv", index=False)

In [ ]:
def get_artist_code(artist_name):
    conn = sqlite3.connect('info.db')
    c = conn.cursor()
    c.execute("SELECT code FROM artists WHERE name=:name", {'name': artist_name})
    return c.fetchone()


def get_artist_name(artist_code):
    conn = sqlite3.connect('info.db')
    c = conn.cursor()
    c.execute("SELECT name FROM artists WHERE code=:code", {'code': artist_code})
    return c.fetchone()

In [ ]:

c.execute("SELECT * FROM artists")
c.fetchall()